In [1]:
# Marta Matice Valdman 1 - solution
# here is the original file in OneDrive
import scipy.io as io
import numpy as np


# from scipy.io import loadmat  # this is the SciPy module that loads mat-files
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd

import numpy.linalg as la
import scipy.linalg as spla

import time
from multiprocessing import Pool, cpu_count
# from functools import partial
import collections
import os
import math

n_processors = cpu_count()

# read file
file = 'mesh_3D_level5.mat'
coordinates = 'coords'
elements = 'elems'

mesh = mesh_coordinates = io.loadmat(file)
mesh_coordinates = mesh['coords']
mesh_elements = mesh['elems']
n_n = mesh_coordinates.shape[0]        # number of nodes
n_e = mesh_elements.shape[0]           # number of element
dim = mesh_coordinates.shape[1]        # dimension
print(f'no.nodes: {n_n}, no.elements: {n_e}, dim: {dim}')


no.nodes: 48385, no.elements: 267960, dim: 3


In [2]:
mesh

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Nov  3 10:45:39 2021',
 '__version__': '1.0',
 '__globals__': [],
 'coords': array([[-0.5       , -0.5       ,  0.        ],
        [ 0.5       , -0.5       ,  0.        ],
        [ 0.5       ,  0.5       ,  0.        ],
        ...,
        [-0.03878742, -0.06282786,  0.45052623],
        [-0.0870318 ,  0.10938944,  0.41051007],
        [ 0.08577658, -0.01182843,  0.42937678]]),
 'elems': array([[48129, 47651, 47891, 47939],
        [45730, 43627, 44496, 44558],
        [45187, 42423, 43185, 43982],
        ...,
        [44563, 44381, 42655, 43525],
        [44563, 43525, 43892, 44381],
        [46333, 45223, 44686, 44989]], dtype=uint16)}

In [3]:

# print(mesh_coordinates.shape)
# print(mesh_elements.shape)

# coords 3D matice
# single process

start_time = time.time()

coords3D = np.zeros((dim, dim, n_e))
det_coords3D = np.zeros(n_e)
inv_coords3D = np.zeros((dim, dim, n_e))

for el in range(0, n_e):
    nodes = mesh_elements[el]
    coordinates = mesh_coordinates[nodes - 1, :]
    for comp in range(dim):
        coords3D[comp, :, el] = coordinates[comp + 1, :] - coordinates[0, :]

    det_coords3D[el] = np.linalg.det(coords3D[:, :, el])
    inv_coords3D[:, :, el] = np.linalg.inv(coords3D[:, :, el])

print(det_coords3D)
print(sum(det_coords3D) / 6)
print("--single process - %s seconds ---" % (time.time() - start_time))


[2.97979075e-05 2.24306000e-05 2.48574358e-05 ... 2.93612564e-05
 3.12613323e-05 1.90605880e-05]
0.9999999999999877
--single process - 308.0376477241516 seconds ---


In [4]:

# multiprocessing
# multi process


def create_3D_set(i):
    coords3D_1 = (mesh_coordinates[mesh_elements[:, i]] - mesh_coordinates[mesh_elements[:, 0]]).T
    # print(i,coords3D_1.shape)
    return coords3D_1


start = time.time()
pool = Pool(processes=8)

# coords3D = np.array(pool.map(create_3D_set,coords3D))
coords3D = np.array(pool.map(create_3D_set, [i for i in range(1, dim + 1)]))

det_coords3D = np.linalg.det(coords3D.T)
inv_coords3D = np.linalg.inv(coords3D.T)

end = time.time()

print("--mutliprocess - %s seconds ---" % (time.time() - start_time))


IndexError: index 48385 is out of bounds for axis 0 with size 48385